# Phase 4 — Event-level evaluation + Multi-label training + Robustness package

Goal of Phase 4:
1) Convert your **pair-level** table into **event-level** predictions (no “multiple pairs per event” confusion)
2) Train and evaluate **multiple HLT labels** (1 model per label) with the SAME GroupKFold splits
3) Add robustness: calibration, threshold selection, drift vs run/lumi, and clean artifacts export

Input: `/kaggle/working/parquet_dimuon/*.parquet` (from Phase 2)

Output (saved to `/kaggle/working/phase4_artifacts/`):
- `metrics_per_label.csv` (pair-level + event-level)
- `thresholds.csv` (operating points)
- `models/` (joblib models per label)
- `plots/` (ROC/PR, stability, calibration)
- `config.json`


In [1]:
# Cell 1 — Install deps
!pip -q install lightgbm scikit-learn shap pyarrow fastparquet matplotlib seaborn joblib


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 29.0 MB/s eta 0:00:00a 0:00:01


In [ ]:
# Cell 2 — Imports & config
from pathlib import Path
import glob, json
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import GroupKFold
from sklearn.metrics import (
    roc_auc_score, average_precision_score,
    roc_curve, precision_recall_curve,
    brier_score_loss
)
from sklearn.calibration import calibration_curve
import lightgbm as lgb
import joblib

SEED = 42
np.random.seed(SEED)

PARQUET_DIR = Path("/kaggle/input/datasets/katakuricharlotte/parquet-triggeremu/parquet_dimuon")
OUT = Path("/kaggle/working/phase4_artifacts")
(OUT / "models").mkdir(parents=True, exist_ok=True)
(OUT / "plots").mkdir(parents=True, exist_ok=True)

parquet_files = sorted(glob.glob(str(PARQUET_DIR / "*.parquet")))
len(parquet_files), parquet_files[:3]


In [ ]:
# Cell 3 — Load Parquet (scale-friendly)
# Start with all shards; if too big, cap MAX_FILES.
MAX_FILES = None  # set e.g. 5 if you want faster iteration
use_files = parquet_files if MAX_FILES is None else parquet_files[:MAX_FILES]

df = pd.concat([pd.read_parquet(p) for p in use_files], ignore_index=True)
df.shape, df.head(3)


In [ ]:
# Cell 4 — Basic data validation
required_cols = {"run","lumi","event"}
assert required_cols.issubset(df.columns), f"Missing keys: {required_cols - set(df.columns)}"

LABELS = sorted([c for c in df.columns if c.startswith("HLT_")])
assert len(LABELS) > 0, "No HLT labels found."

# event_id for grouping
df["event_id"] = df["run"].astype(str) + ":" + df["lumi"].astype(str) + ":" + df["event"].astype(str)

print("rows:", len(df))
print("unique events:", df["event_id"].nunique())
print("labels:", LABELS[:5], "... total:", len(LABELS))

(df[LABELS].mean().sort_values(ascending=False).head(10))


## Phase 4-A: Define features (explainable & stable)

We keep features that are:
- physically meaningful (m_mumu, pt_mumu, dR, etc.)
- simple context (PV_npvs, MET_pt)
No IDs and no HLT columns.


In [ ]:
# Cell 5 — Feature set and simple preprocessing
drop_cols = set(["run","lumi","event","event_id"]) | set(LABELS)
features = [c for c in df.columns if c not in drop_cols]

X = df[features].copy()
for c in X.columns:
    if X[c].dtype == "object":
        X[c] = pd.to_numeric(X[c], errors="coerce")

# Simple median fill for baseline robustness
X = X.fillna(X.median(numeric_only=True))

X.shape, features


In [ ]:
# Cell 6 — Utility: metrics at pair-level and event-level
def safe_auc(y_true, y_score):
    # Handle edge case where a fold has only one class
    if len(np.unique(y_true)) < 2:
        return np.nan
    return roc_auc_score(y_true, y_score)

def compute_metrics(y_true, y_score):
    return {
        "roc_auc": safe_auc(y_true, y_score),
        "ap": average_precision_score(y_true, y_score),
        "brier": brier_score_loss(y_true, y_score),
        "pos_rate": float(np.mean(y_true)),
        "n": int(len(y_true)),
    }

def event_level_aggregate(df_part, score_col="score", label_col="y"):
    # Aggregate pair rows to event-level:
    # - event score = max(score) (if any pair "looks like" it should fire, event fires)
    # - event label = max(label) (event-level label repeated, so max is safe)
    g = df_part.groupby("event_id", as_index=False).agg(
        y=(label_col, "max"),
        score=(score_col, "max"),
        run=("run", "first"),
        lumi=("lumi", "first"),
        event=("event", "first"),
    )
    return g


## Phase 4-B: Consistent GroupKFold splits (by event)

We split by `event_id` so that the same collision event never appears in both train and validation.


In [ ]:
# Cell 7 — Prepare GroupKFold splits on unique events (stronger than splitting pair rows)
events_unique = df[["event_id"]].drop_duplicates().reset_index(drop=True)
event_ids = events_unique["event_id"].values

gkf = GroupKFold(n_splits=5)

splits = list(gkf.split(event_ids, np.zeros(len(event_ids)), groups=event_ids))
len(splits), [ (len(tr), len(va)) for tr,va in splits ]


In [ ]:
# Cell 8 — Map event splits to pair-row indices (so training uses pair rows but validation stays event-disjoint)
event_to_fold = {}
for fold, (_, va_idx) in enumerate(splits, start=1):
    for eid in event_ids[va_idx]:
        event_to_fold[eid] = fold

df["fold"] = df["event_id"].map(event_to_fold).astype(int)
df["fold"].value_counts().sort_index()


## Phase 4-C: Train 1 model per label (with class imbalance handling)

We train LightGBM models with `scale_pos_weight` for imbalance, and we store:
- per-fold metrics
- overall OOF predictions
- event-level metrics after aggregating pair scores to event scores


In [ ]:
# Cell 9 — Training loop for multiple labels
params = dict(
    n_estimators=800,
    learning_rate=0.05,
    num_leaves=31,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=SEED,
    n_jobs=-1,
)

all_metrics = []
oof_scores = {}   # label -> oof scores (pair-level)
models = {}

for label in LABELS:
    y = df[label].astype(int).values

    # Skip degenerate labels
    if y.mean() < 1e-4 or y.mean() > 1 - 1e-4:
        all_metrics.append({"label": label, "note": "skipped_degenerate", "pos_rate": float(y.mean())})
        continue

    oof = np.zeros(len(df), dtype=float)

    for fold in sorted(df["fold"].unique()):
        tr_mask = df["fold"].values != fold
        va_mask = df["fold"].values == fold

        X_tr, X_va = X.loc[tr_mask], X.loc[va_mask]
        y_tr, y_va = y[tr_mask], y[va_mask]

        pos = max(y_tr.sum(), 1)
        neg = max(len(y_tr) - y_tr.sum(), 1)
        spw = neg / pos

        clf = lgb.LGBMClassifier(objective="binary", scale_pos_weight=spw, **params)
        clf.fit(X_tr, y_tr)

        pred = clf.predict_proba(X_va)[:, 1]
        oof[va_mask] = pred

        m_pair = compute_metrics(y_va, pred)
        all_metrics.append({
            "label": label,
            "fold": int(fold),
            "level": "pair",
            "scale_pos_weight": float(spw),
            **m_pair
        })

    # Store OOF
    oof_scores[label] = oof

    # Train final model on all data
    pos = max(y.sum(), 1)
    neg = max(len(y) - y.sum(), 1)
    spw = neg / pos
    final_model = lgb.LGBMClassifier(objective="binary", scale_pos_weight=spw, **params)
    final_model.fit(X, y)

    models[label] = final_model
    joblib.dump(final_model, OUT / "models" / f"{label}.joblib")

len(models), list(models.keys())[:5]


In [ ]:
# Cell 10 — Compute event-level metrics from OOF (per label)
event_metrics = []

for label, oof in oof_scores.items():
    tmp = df[["event_id","run","lumi","event"]].copy()
    tmp["y"] = df[label].astype(int).values
    tmp["score"] = oof

    ev = event_level_aggregate(tmp, score_col="score", label_col="y")
    m_ev = compute_metrics(ev["y"].values, ev["score"].values)

    event_metrics.append({"label": label, "level": "event", "fold": "OOF", **m_ev})

event_metrics_df = pd.DataFrame(event_metrics).sort_values(["ap"], ascending=False)
event_metrics_df.head(15)


In [ ]:
# Cell 11 — Combine and save metrics
metrics_df = pd.DataFrame(all_metrics)
metrics_out = OUT / "metrics_per_label.csv"
metrics_df.to_csv(metrics_out, index=False)

event_out = OUT / "event_metrics_oof.csv"
event_metrics_df.to_csv(event_out, index=False)

metrics_out, event_out


## Phase 4-D: Choose operating thresholds (engineering deliverable)

We pick a threshold for each label based on a target **recall** (efficiency) or based on maximizing F1.
This makes the model usable like a trigger/selection component.


In [ ]:
# Cell 12 — Threshold selection (OOF) per label
def choose_threshold(y_true, y_score, target_recall=0.95):
    prec, rec, thr = precision_recall_curve(y_true, y_score)
    # precision_recall_curve returns thr of length n-1
    # Find first threshold reaching recall >= target_recall
    idx = np.where(rec[:-1] >= target_recall)[0]
    if len(idx) == 0:
        # fallback: best F1
        f1 = 2 * (prec * rec) / (prec + rec + 1e-12)
        best = np.nanargmax(f1)
        return float(thr[max(best-1, 0)]), float(prec[best]), float(rec[best])
    i = idx[0]
    return float(thr[i]), float(prec[i]), float(rec[i])

threshold_rows = []
for label, oof in oof_scores.items():
    tmp = df[["event_id"]].copy()
    tmp["y"] = df[label].astype(int).values
    tmp["score"] = oof
    ev = event_level_aggregate(tmp, score_col="score", label_col="y")

    th, p, r = choose_threshold(ev["y"].values, ev["score"].values, target_recall=0.95)
    threshold_rows.append({"label": label, "threshold": th, "precision_at_th": p, "recall_at_th": r, "event_pos_rate": float(ev["y"].mean())})

thresholds_df = pd.DataFrame(threshold_rows).sort_values("event_pos_rate", ascending=False)
thresholds_df


In [ ]:
# Cell 13 — Save thresholds
thr_out = OUT / "thresholds.csv"
thresholds_df.to_csv(thr_out, index=False)
thr_out


## Phase 4-E: Plots (ROC/PR + calibration) for top labels
We generate plots for the top 3 labels by event-level AP (OOF).
